Импортируем конфиг и загружаем данные


In [ ]:
import pandas as pd
import yaml

with open("../../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)


df = pd.read_csv(cfg["mental_health"]["train_dataset"])
df_test = pd.read_csv(cfg["mental_health"]["test_dataset"])

In [ ]:
df.columns

Производим предварительную обработку данных\
Разделяем признаки и целевую переменную\
Информация об имени человека не нужна


In [ ]:
df_copy = df.copy()
df_test_copy = df_test.copy()

df_copy = df_copy.drop(["Name"], axis=1)
df_test_copy = df_test_copy.drop(["Name"], axis=1)

X = df_copy.drop(["Depression", "id"], axis=1)
X_test = df_test_copy.drop(["id"], axis=1)
y = df_copy["Depression"]

Определяем числовые и категориальные признаки


In [ ]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["int64", "float64"]).columns.tolist()

Создаем конвейеры обработки данных\
Для числовых признаков пропущенные значения заполняются медианным значением\
Для категориальных признаков значения заполняются наиболее частым значением\
Переменные категориальных признаков кодируются в числовой формат OneHotEncoder


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


num_pipe = Pipeline(
    [("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

cat_pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

Создаем общий обработчик числовых и категориальных признаков


In [ ]:
transformer = ColumnTransformer(
    [("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)]
)

Создаем словарь с различными моделями классификации для сравнения эффективности


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


models = {
    "Logistic Regression": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
}

Проводим кросс-валидацию для выбора лучшей модели\
Для каждой модели создаем пайплайн (предобработка и модель)\
Вычисляем метрики и выбираем лучшую модель


In [ ]:
from sklearn.model_selection import cross_val_score


best_model = None
best_score = 0

for name, model in models.items():
    pipe = Pipeline([("transform", transformer), ("model", model)])
    scores = cross_val_score(pipe, X, y, cv=4, scoring="f1")
    mean_score = scores.mean()
    print(f"{name:<20}: {mean_score:.4f}")

    if mean_score > best_score:
        best_score = mean_score
        best_model = pipe

Обучаем лучшую модель на всех тренировочных данных\
Делаем предсказания для тестовых данных и сохраняем результат


In [ ]:
best_model.fit(X, y)
pred = best_model.predict(X_test)

output = pd.DataFrame({"id": df_test["id"], "Depression": pred.astype(int)})
output.to_csv("output.csv", index=False)